## João Pedro Rodrigues Freitas - 11316552

### TODO
- Fazer os mini batches (train, test)
- Fazer o one-hot
- conferir normalização
- Tirar a seed
- Não passar func ativ na ultima camada
- Fazer o espelhamento automático

In [155]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

## Data

In [156]:
class Data():
    def __init__(self, X: np.ndarray) -> None:
        self.data = X

    def getData(self) -> np.ndarray:
        return self.data

    def setTrainRatio(self, ratio: float) -> None:
        self.train_ratio = ratio

    def getTrainRatio(self) -> float:
        return self.train_ratio
    
    def getAttrSize(self) -> int:
        return self.data.shape[1]
    
    def shuffleData(self) -> None:
        self.data = np.random.permutation(self.data)

    def getTrainData(self) -> np.ndarray:
        return self.data[:int(self.data.shape[0] * self.train_ratio)]
    
    def getTestData(self) -> np.ndarray:
        return self.data[int(self.data.shape[0] * self.train_ratio):]

    def normalize(self) -> None:
        '''Normaliza cada atributo para o intervalo [0, 1]'''
        for i in range(self.getAttrSize()):
            self.data[:,i] = (self.data[:,i] - np.min(self.data[:,i])) / (np.max(self.data[:,i]) - np.min(self.data[:,i]))

## Layer

In [157]:
class Layer():
    def __init__(self, n_neurons: int, actv_func, inputs = None, lastLayer = False) -> None:
        self.n_neurons = n_neurons
        self.W = None
        self.b = None

        self.inputs = inputs
        self.shape = None
        self.inputShape = None

        self.actv_func = actv_func

        self.out = None
        self.actv = None

        self.lastLayer = lastLayer

        self._setWeights()

    def _setWeights(self) -> None:  
        if self.W is None and self.inputs is not None:
            self.inputShape = self.inputs.shape

            self.shape = (self.inputShape[-1], self.n_neurons)
            # Inicializa os pesos e bias com distribuição uniforme
            # entre -0.5 e 0.5
            self.W = np.random.rand(self.shape[0], self.shape[1]) - 0.5
            self.b = np.random.rand(self.n_neurons) - 0.5

    def process(self, inputs):
        '''Forward da camada'''
        self.inputs = inputs
        self._setWeights()
        self.out = np.dot(self.inputs, self.W) + self.b
        if not self.lastLayer:
            self.actv = self.actv_func(self.out)
        else:
            self.actv = self.out

        return self

    def getOutput(self):
        return self.out
    
    def getActivation(self):
        return self.actv
        


# Autoencoder

In [158]:
class Autoencoder():
    def __init__(self) -> None:
        self.layers = np.array([], dtype=Layer)

        self.epochs = None
        self.lr = None

        self.inputs = None # X
        self.targets = None # X
        self.outputs = [] # X_hat

    def addLayer(self, layer: Layer) -> None:
        self.layers = np.append(self.layers, layer)

    # TODO: separar entre encoder e decoder
    def forward(self, inputs):
        self.outputs = []
        self.outputs.append(inputs)

        for layer in self.layers:
            inputs = layer.process(inputs).getActivation()
            self.outputs.append(inputs)

            # inputs = layer.getActivation()

        return inputs
    
    # TODO: separar entre encoder e decoder
    def backward(self) -> None:
        i = self.layers.size

        x_hat = self.outputs[-1]
        err = 2 * (self.targets - x_hat)

        # TODO: dividir por n_samples

        for layer, x_hat in zip(self.layers[::-1], self.outputs[::-1]):
            delta = err

            if (i != self.layers.size):
                delta *= layer.actv_func(x_hat, derivative=True)

            # delta = err * layer.actv_func(x_hat, derivative=True)

            err = np.dot(delta, layer.W.T)

            if i > 0:
                prev_output = self.outputs[i-1]
                # dW = np.dot(prev_output.T, delta) / n
                # db = np.sum(delta, axis=0) / n
                dW = np.dot(prev_output.T, delta)
                db = delta.mean()

                layer.W += self.lr * dW
                layer.b += self.lr * db


            i -= 1

    # # TODO: separar entre encoder e decoder
    # def backward(self) -> None:
    #     x_hat = self.outputs[-1]
    #     err = 2 * (self.targets - x_hat)
    #     i = self.layers.size 

    #     # TODO: dividir por n_samples

    #     for layer, x_hat in zip(self.layers[::-1], self.outputs[::-1]):
    #         delta = err

    #         if (i != self.layers.size):
    #             delta *= layer.actv_func(x_hat, derivative=True)

    #         # delta = err * layer.actv_func(x_hat, derivative=True)

    #         err = np.dot(delta, layer.W.T)

    #         if i > 0:
    #             prev_output = self.outputs[i-1]
    #             # dW = np.dot(prev_output.T, delta) / n
    #             # db = np.sum(delta, axis=0) / n
    #             dW = np.dot(prev_output.T, delta)
    #             db = delta.mean()

    #             layer.W += self.lr * dW
    #             layer.b += self.lr * db


    #         i -= 1
        
    def fit(self, inputs, targets, lr: float = 0.01, epochs: int = 100) -> None:
        self.inputs = inputs
        self.targets = targets
        self.lr = lr

        errors = []

        for epoch in range(epochs):
            self.forward(inputs)
            # error = abs(np.ravel(self.targets - self.outputs[-1])).mean()
            # errors.append(error)
            # report_progress(epoch, epochs-1,error)
            self.backward()

            if epoch % 10 == 0:
                print(f'Epoch: {epoch}')
                print(f'Loss: {np.mean(np.square(self.targets - self.outputs[-1]))}')


    def predict(self, inputs) -> np.ndarray:
        self.forward(inputs)
        return self.outputs[-1]


In [159]:
def sigmoid(x, derivative=False):
    if derivative:
        return sigmoid(x) * (1 - sigmoid(x))
    return 1 / (1 + np.exp(-x))

def tanh(x, derivative=False):
    if derivative:
        return 1 - np.tanh(x)**2
    return np.tanh(x)

def createModel(layers, actv_func):
    model = Autoencoder()

    # for l in layers[1:]:
    #     layer = Layer(l, actv_func)
    #     model.addLayer(layer)

    for i in range(1, len(layers)):
        layer = Layer(layers[i], actv_func, lastLayer = (i == len(layers)-1))
        model.addLayer(layer)

    return model

## Main

In [160]:
def main():
    X = np.vstack([np.random.normal(1, 0.5, [100,64]),
                   np.random.normal(-2, 1, [100,64]),
                   np.random.normal(3, 0.75, [100,64])])
    
    data = Data(X)

    data.normalize()
    data.shuffleData()
    data.setTrainRatio(0.8)

    inputDim = data.getAttrSize()
    hiddenLayers = 2

    # Número de neurônios em cada camada escondida
    # A última camada dessa lista é a dimensão latente
    neuronsPerLayer = [32, 16]
    # [5, 3, 2, 3, 5]

    # TODO: fazer espelhar a rede
    teste = [32, 16, 32]

    autoencoder = createModel([inputDim] + teste + [inputDim], actv_func=sigmoid)
    autoencoder.fit(data.getTrainData(), data.getTrainData(), lr=0.01, epochs=100)


In [161]:
if __name__ == "__main__":
    main()

Epoch: 0
Loss: 1.0608244165545557
Epoch: 10
Loss: 0.4084111700687896
Epoch: 20
Loss: 0.3134902828251414
Epoch: 30
Loss: 0.27277834126889794
Epoch: 40
Loss: 2370.5456821758116
Epoch: 50
Loss: 0.12298787616753062
Epoch: 60
Loss: 0.12293941649206362
Epoch: 70
Loss: 0.12290706442641314
Epoch: 80
Loss: 0.12288546592948216
Epoch: 90
Loss: 0.12287104660075311


/tmp/ipykernel_121215/2027884050.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
